In [160]:
import pandas as pd

fa = pd.read_csv("facialexpression.csv")
glasses = pd.read_csv("labels_glasses.csv", header = 0)
race = pd.read_csv("race.csv")

In [161]:
race.shape

(64441, 3)

In [162]:
# generate a dictionary of replacement values
replacements = {}
for i in range(fa.shape[0]):
    replacements[i] = f'{i:05}.png'

# replace values in the "name" column
fa['Name'] = fa['Name'].replace(replacements)

# display the modified dataframe
fa.head()


,Name,FacialExp
0,00000.png,neutral
1,00001.png,happy
2,00002.png,neutral
3,00003.png,neutral
4,00004.png,neutral


In [163]:
merged_df = fa.merge(glasses, on="Name", how="inner")
print(merged_df.shape)
# merge the race dataframe with the selected_df on the "Name" column
merged_df = merged_df.merge(race, on="Name", how="inner")
print(merged_df.shape)

(70000, 3)
(64441, 5)


In [164]:
# merged_df.groupby(["FacialExp"]).count()
merged_df.FacialExp = merged_df.FacialExp.replace({"angry" : "neg", "disgust": "neg", "fear": "neg", "happy" : "pos", "sad": "neg", "surprise" : "pos"})
merged_df.Race = merged_df.Race.replace({"indian" : "asian", "middle eastern": "asian"})

In [165]:
df_final = merged_df.loc[merged_df["FacialExp"] == "neg"]

In [166]:
black = merged_df.loc[merged_df["Race"] == "black"]
hos = merged_df.loc[merged_df["Race"] == "latino hispanic"]


In [167]:
df_final = pd.concat([df_final, black])
df_final = pd.concat([df_final, hos])

In [168]:
df_final = df_final.drop_duplicates(subset="Name")

In [171]:
women_images = list(df_final.loc[df_final["Gender"] == "Woman"]["Name"])
df_final_other_women = merged_df.loc[~merged_df["Name"].isin(women_images)]
df_final_other_women = df_final_other_women.loc[df_final_other_women["Gender"] == "Woman"][:5001]

In [180]:
asian_images = list(df_final.loc[df_final["Race"] == "asian"]["Name"])
df_final_other_asian = merged_df.loc[~merged_df["Name"].isin(asian_images)]
df_final_other_asian = df_final_other_asian.loc[df_final_other_asian["Race"] == "asian"][:5001]

In [181]:
df_final = pd.concat([df_final, df_final_other_asian])
df_final = df_final.drop_duplicates(subset="Name")

In [173]:
df_final = pd.concat([df_final, df_final_other_women])
df_final = df_final.drop_duplicates(subset="Name")

In [182]:
df_final.shape

(25586, 5)

In [183]:
df_final.groupby(["Race"]).count()

,Name,FacialExp,Glasses,Gender
Race,,,,
asian,7750,7750,7750,7750
black,2717,2717,2717,2717
latino hispanic,7305,7305,7305,7305
white,7814,7814,7814,7814


In [184]:
df_final.groupby(["FacialExp"]).count()

,Name,Glasses,Gender,Race
FacialExp,,,,
neg,6523,6523,6523,6523
neutral,5769,5769,5769,5769
pos,13294,13294,13294,13294


In [185]:
df_final.groupby(["Glasses"]).count()

,Name,FacialExp,Gender,Race
Glasses,,,,
0,17064,17064,17064,17064
1,8517,8517,8517,8517
No face detected,5,5,5,5


In [186]:
df_final.groupby(["Gender"]).count()

,Name,FacialExp,Glasses,Race
Gender,,,,
Man,14983,14983,14983,14983
Woman,10603,10603,10603,10603


In [187]:
df_final.to_csv('df_final.csv')

In [188]:
df_final.shape

(25586, 5)

In [12]:

# drop rows with "No face detected" in the "Glasses" column
merged_df["Glasses"].replace("No face detected", 0, inplace=True)

# group the merged dataframe by FacialExp and Glasses
grouped_df = merged_df.groupby(["FacialExp", "Glasses", 'Gender', 'Race'])

# create an empty list to store the selected indices
selected_indices = []

# iterate over each group
for name, group in grouped_df:
    # calculate the number of pictures to select for this group
    n = min(25000 // len(grouped_df), len(group))
    # randomly select n indices from the group
    indices = group.sample(n).index.tolist()
    # add the selected indices to the list
    selected_indices.extend(indices)

# use the selected indices to create a new dataframe with the selected pictures
selected_df = merged_df.loc[selected_indices]

# display the number of pictures in each category
print(selected_df.groupby(["FacialExp", "Glasses", "Gender", "Race"]).size())

# display the selected dataframe
print(selected_df)

FacialExp  Glasses  Gender  Race           
angry      0        Woman   asian                1
                    Man     asian              160
                            black               41
                            indian              17
                            latino hispanic    160
                                              ... 
surprise   1        Woman   black                1
                            indian               2
                            latino hispanic     10
                            middle eastern       2
                            white               64
Length: 156, dtype: int64
            Name FacialExp Glasses Gender   Race
21697  27256.png     angry       0  Woman  asian
64114  69673.png     angry       0    Man  asian
45247  50806.png     angry       0    Man  asian
32466  38025.png     angry       0    Man  asian
62726  68285.png     angry       0    Man  asian
...          ...       ...     ...    ...    ...
3637   03637.png  surprise

In [18]:
merged_df.to_csv("merge_df.csv")

In [24]:
selected_df.shape

(11397, 5)

FacialExp  Glasses
angry      0          1785
           1          1785
disgust    0             1
           1             4
fear       0           403
           1           397
happy      0          1785
           1          1785
neutral    0          1785
           1          1785
sad        0          1377
           1           746
surprise   0           932
           1           310
dtype: int64
            Name FacialExp Glasses
61922  61922.png     angry       0
20846  20846.png     angry       0
16835  16835.png     angry       0
6771   06771.png     angry       0
46941  46941.png     angry       0
...          ...       ...     ...
12070  12070.png  surprise       1
10382  10382.png  surprise       1
68622  68622.png  surprise       1
63819  63819.png  surprise       1
36234  36234.png  surprise       1

[14880 rows x 3 columns]


In [22]:
merged_df.head()

,Name,FacialExp,Glasses,Gender,Race
0,00000.png,neutral,0,Woman,asian
1,00001.png,happy,1,Man,white
2,00002.png,neutral,0,Woman,black
3,00003.png,neutral,0,Man,asian
4,00004.png,neutral,0,Man,latino hispanic


In [1]:
import boto3
import os 

AWS_Access_Key_ID = os.environ['AWS_ACCESS_KEY_ID']
AWS_Secret_Access_Key = os.environ['AWS_SECRET_ACCESS_KEY']

session = boto3.Session(
    aws_access_key_id = AWS_Access_Key_ID,
    aws_secret_access_key = AWS_Secret_Access_Key,
    region_name='us-west-2'
)

s3 = session.resource('s3')
for bucket in s3.buckets.all():
    print(bucket.name)

dalle2images
